In [1]:
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia('en')

In [3]:
def get_articles(cat_main):
    
    articles_all = []
    
    for p in cat_main.categorymembers.values():

        if p.namespace == wikipediaapi.Namespace.CATEGORY and ('painter' in p.title or 'Painter' in p.title):
            articles_all.extend(get_articles(p))
        
        elif p.namespace == wikipediaapi.Namespace.MAIN:
            articles_all.append(p)

    return articles_all

In [4]:
def get_df_for_cat_value(value_cat_lst, clean_func, valueName):
    
    pages_all = []
    pages_name_all = []
    value_all = []

    for value in tqdm(value_cat_lst):
        
        pages_oneV = get_articles(value)
        pages_name_oneV = [p.title for p in pages_oneV]

        value_clean = clean_func(value)
        value_oneV = [value_clean for p in pages_oneV]

           
        pages_all.extend(pages_oneV)
        pages_name_all.extend(pages_name_oneV)
        value_all.extend(value_oneV)
    
    print('Number of pages before deleting duplicates: ', len(pages_all))
    
    columns = ['page', 'page name', valueName]
    df = pd.DataFrame(columns=columns) 

    df['page'] = pages_all
    df['page name'] = pages_name_all
    df[valueName] = value_all
    
    df = df.groupby('page name').agg({'page':'first', 
                                       valueName: set}).reset_index()
    
    df[valueName] = df[valueName].apply(list)

    
    return df

## Nationalities

In [5]:
def clean_nationality(nat_cat):
    nat = nat_cat.title
    
    nat = nat.split('Category:')[1]
    
    if 'from Georgia' in nat:
        return 'Georgian'
    if 'from the Principality' in nat:
        return 'Liège'
    return nat.split(' painters')[0]

In [6]:
nationalities_cat_lst = []

for nat in  wiki_wiki.page("Category:Painters_by_nationality").categorymembers.values():
    nationalities_cat_lst.append(nat)
    
nationalities_cat_lst = nationalities_cat_lst[8:]

In [7]:
df_nationality = get_df_for_cat_value(value_cat_lst=nationalities_cat_lst, 
                                      clean_func=clean_nationality, 
                                      valueName='nationality')

100%|████████████████████████████████████████████████████████████████████████████████| 170/170 [05:19<00:00,  1.88s/it]


Number of pages before deleting duplicates:  131442


In [8]:
df_nationality

,page name,page,nationality
0,'Abd al-Hayy,"'Abd al-Hayy (id: ??, ns: 0)","[Persian, Iranian]"
1,'the other' Jan van Kessel,"'the other' Jan van Kessel (id: ??, ns: 0)",[Flemish]
2,108 (artist),"108 (artist) (id: ??, ns: 0)",[Italian]
3,3Steps,"3Steps (id: ??, ns: 0)",[German]
4,A. A. Raiba,"A. A. Raiba (id: ??, ns: 0)",[Indian]
...,...,...,...
37045,Živorad Nastasijević,"Živorad Nastasijević (id: ??, ns: 0)",[Serbian]
37046,Ștefan Câlția,"Ștefan Câlția (id: ??, ns: 0)",[Romanian]
37047,Ștefan Dimitrescu,"Ștefan Dimitrescu (id: ??, ns: 0)",[Romanian]
37048,Ștefan Luchian,"Ștefan Luchian (id: ??, ns: 0)",[Romanian]


## Dates and Other Info

In [9]:
def get_dates(page):
    cats = page.categories.keys()

    death_year, birth_year = None, None
    
    for c in cats:

        if 'births' in c:
            match = re.search(r'\d{4}', c) 
            if match: birth_year = match[0]

        if 'deaths' in c:
            match = re.search(r'\d{4}', c) 
            if match: death_year = match[0]

    return [birth_year, death_year, page.fullurl, list(cats)]

In [20]:

    births = []
    deaths = []
    urls = []
    categories_all = []


    for p in tqdm(df_nationality[21719:]['page']):
        b, d, u, cs = get_dates(p)

        births.append(b)
        deaths.append(d)
        urls.append(u)
        categories_all.append(cs)
        
        

100%|██████████████████████████████████████████████████████████████████████████| 15331/15331 [1:41:27<00:00,  2.52it/s]


In [21]:
len(births)

15331

## Other Info

# Save

In [11]:
def clean_title(title):
    if ' (' in title:
        return title.split(' (')[0]
    return title

In [22]:
columns = ['page', 'page name', 'painter name', 'url', 'birth', 'death', 'nationality', 'categories']
df = pd.DataFrame(columns=columns)


df['page'] = df_nationality['page']
df['page name'] = df_nationality['page name']
df['painter name'] = df_nationality['page name'].apply(clean_title)
df['url'] = urls
df['birth'] = births
df['death'] = deaths
df['nationality'] = df_nationality['nationality']
df['categories'] = categories_all


df

,page,page name,painter name,url,birth,death,nationality,categories
21719,"Katharina Rapp (id: 1628554, ns: 0)",Katharina Rapp,Katharina Rapp,https://en.wikipedia.org/wiki/Katharina_Rapp,1948,None,[German],"[Category:1948 births, Category:20th-century G..."
21720,"Katharine Carl (id: 8544717, ns: 0)",Katharine Carl,Katharine Carl,https://en.wikipedia.org/wiki/Katharine_Carl,1865,1938,[American],"[Category:1865 births, Category:1938 deaths, C..."
21721,"Katharine Church (id: 56389959, ns: 0)",Katharine Church,Katharine Church,https://en.wikipedia.org/wiki/Katharine_Church,1910,1999,[British],"[Category:1910 births, Category:1999 deaths, C..."
21722,"Katharine Emma Maltwood (id: 10726642, ns: 0)",Katharine Emma Maltwood,Katharine Emma Maltwood,https://en.wikipedia.org/wiki/Katharine_Emma_M...,1878,1961,[British],"[Category:1878 births, Category:1961 deaths, C..."
21723,"Katharine Lane Weems (id: 11172863, ns: 0)",Katharine Lane Weems,Katharine Lane Weems,https://en.wikipedia.org/wiki/Katharine_Lane_W...,1899,1989,[American],"[Category:1899 births, Category:1989 deaths, C..."
...,...,...,...,...,...,...,...,...
37045,"Živorad Nastasijević (id: 62254857, ns: 0)",Živorad Nastasijević,Živorad Nastasijević,https://en.wikipedia.org/wiki/%C5%BDivorad_Nas...,1893,1966,[Serbian],"[Category:1893 births, Category:1966 deaths, C..."
37046,"Ștefan Câlția (id: 13409891, ns: 0)",Ștefan Câlția,Ștefan Câlția,https://en.wikipedia.org/wiki/%C8%98tefan_C%C3...,1942,None,[Romanian],"[Category:1942 births, Category:All stub artic..."
37047,"Ștefan Dimitrescu (id: 6000470, ns: 0)",Ștefan Dimitrescu,Ștefan Dimitrescu,https://en.wikipedia.org/wiki/%C8%98tefan_Dimi...,1886,1933,[Romanian],"[Category:1886 births, Category:1933 deaths, C..."
37048,"Ștefan Luchian (id: 2475912, ns: 0)",Ștefan Luchian,Ștefan Luchian,https://en.wikipedia.org/wiki/%C8%98tefan_Luchian,1868,1917,[Romanian],"[Category:1868 births, Category:1917 deaths, C..."


In [23]:
df.to_csv('Data_GoogleArts_V1.csv')

## Load Test

In [10]:
df_tmp = pd.read_csv('Data_GoogleArts_V1_0_21719.csv', index_col=0)
df_tmp.head()

,page,page name,painter name,url,birth,death,nationality,categories
0,"'Abd al-Hayy (id: 39316086, ns: 0)",'Abd al-Hayy,'Abd al-Hayy,https://en.wikipedia.org/wiki/%27Abd_al-Hayy,1374.0,1405.0,"['Persian', 'Iranian']","['Category:1374 births', 'Category:1405 deaths..."
1,"'the other' Jan van Kessel (id: 52935555, ns: 0)",'the other' Jan van Kessel,'the other' Jan van Kessel,https://en.wikipedia.org/wiki/%27the_other%27_...,1620.0,NaN,['Flemish'],"['Category:1620 births', 'Category:Animal arti..."
2,"108 (artist) (id: 18446241, ns: 0)",108 (artist),108,https://en.wikipedia.org/wiki/108_(artist),1978.0,NaN,['Italian'],"['Category:1978 births', 'Category:20th-centur..."
3,"3Steps (id: 45689974, ns: 0)",3Steps,3Steps,https://en.wikipedia.org/wiki/3Steps,NaN,NaN,['German'],"['Category:Articles with hCards', 'Category:Ge..."
4,"A. A. Raiba (id: 5402895, ns: 0)",A. A. Raiba,A. A. Raiba,https://en.wikipedia.org/wiki/A._A._Raiba,1922.0,2016.0,['Indian'],"['Category:1922 births', 'Category:2016 deaths..."


In [32]:
wiki_wiki = wikipediaapi.Wikipedia('en',extract_format=wikipediaapi.ExtractFormat.WIKI)


"3Steps (pronounced:  θriː stɛps) is a German-based contemporary artist collective between the twins Kai Harald Krieger and Uwe Harald Krieger (born March 15, 1980) and Joachim Pitt (born December 8, 1980).\nThe works of 3Steps have developed from mural art, graffiti art and street art. The spray can is the central media of 3Steps. The collective paints huge images on facades and murals, as well as several kinds of paintable media in the studio. Bright colors and the reflection of a modern society express the intention of the three friends.\n\nIn November 2014 3Steps received the “Kultur- und Kreativpilot Deutschland” (Cultural and Creative pilot of Germany) award by the Federal Republic of Germany. 3Steps lives and works in the University town of Giessen, central Germany.\n\nDevelopment\n3Steps was founded in fall 1998. The style of the collective quickly developed from the classic “New York Style Writing” graffiti and high art graffiti to artistic short stories on large-scale murals.

In [20]:
df_tmp.at[0,'categories']

"['Category:1374 births', 'Category:1405 deaths', 'Category:All articles needing additional references', 'Category:All stub articles', 'Category:Articles needing additional references from July 2016', 'Category:Asian painter stubs', 'Category:Commons category link is on Wikidata', 'Category:Iranian artist stubs', 'Category:Persian miniature painters', 'Category:Wikipedia articles with ULAN identifiers', 'Category:Wikipedia articles with VIAF identifiers', 'Category:Wikipedia articles with WORLDCATID identifiers']"

In [31]:
re.search(str(int(df_tmp.at[1,'birth'])),wiki_wiki.page(df_tmp.at[1,'page name']).text).start()

47

In [3]:
import json

In [79]:
with open('output.json', 'w',encoding='utf-16') as f:
    for j,row in enumerate(df_tmp.iterrows()):
        d = {}
        e = {}
        d['docid'] = "Wikipedia-"+str(j).zfill(5)
        d['doctext'] = wiki_wiki.page(row[1]['page name']).summary
        if np.isnan(row[1]['birth']):
            e['birth'] = []
        else:
            b = re.search(str(int(row[1]['birth'])),d['doctext'])
            if b is not None:
                e['birth'] = [[str(int(row[1]['birth'])),b.start()]]
            else:
                e['birth'] = []
         
        if np.isnan(row[1]['death']):
            e['death'] = []
        else:
            de = re.search(str(int(row[1]['death'])),d['doctext'])
            if de is not None:
                e['death'] = [[str(int(row[1]['death'])),de.start()]]
            else:
                e['death'] = []
            
        l = row[1]['nationality']
        l = l.strip('][').split(", ")
        l_out = []
        for i in l: 
            i = i.strip("'")
            n = re.search(i,d['doctext'])
            if n is not None: 
                l_out.append([i,n.start()])
        e['nationality'] = l_out
        d["extracts"] = e            
        json.dump(d,f,ensure_ascii=False)
        f.write("\n")
        if j%100 == 0:
            print(j)
        

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [4]:
df_tmp = pd.read_csv('Data_GoogleArts_V1_0_21719.csv', index_col=0)
l = df_tmp['painter name'].to_numpy()
g = open('train.json','w',encoding='utf-8')
dev = open('dev.json','w',encoding='utf-8')
test = open('test.json','w',encoding='utf-8')
with open('output.json', 'r',encoding='utf-16') as f:
    c = 0
    m = len(l)
    for i,j in zip(f,l):  
        c+=1
        d = json.loads(i)
        for k in d['extracts'].keys():
            if d['extracts'][k]:
                d['extracts'][k] = [d['extracts'][k]]
        n = re.search(j,d['doctext'])
        d["docid"] = "Wikipedia-"+d["docid"].split("-")[-1]
        if n is not None:
            d['extracts']['painter_name'] = [[[j,n.start()]]]
        else:
            d['extracts']['painter_name'] = [[[j,0]]]
        if c<m*0.9:
            json.dump(d,g,ensure_ascii=False)
            g.write("\n")
        elif c<m*0.95:
            json.dump(d,dev,ensure_ascii=False)
            dev.write("\n")
        elif c<m:
            json.dump(d,test,ensure_ascii=False)
            test.write("\n")
        else:
            break
            
g.close()
dev.close()
test.close()